In [ ]:
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer -q

^C


In [3]:
import torch
import datasets
from datasets import Dataset, DatasetDict
import random
import pandas as pd
from IPython.display import display, HTML

ModuleNotFoundError: No module named 'torch'

## Load in data and convert to a PyTorch Dataset for training

In [2]:
audio_data = datasets.load_dataset("csv", data_files="cv-valid-train.csv")
audio_data

NameError: name 'datasets' is not defined

##### Common Voice dataset provides a lot more information about each audio file such as accent. However, I have decided to only use the `'filename'` and the `'text'` for fine tuning.